In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("kaggle_dataset\marketing_campaign\marketing_campaign.csv", sep=";")

df.head()

<>:4: SyntaxWarning: invalid escape sequence '\m'
<>:4: SyntaxWarning: invalid escape sequence '\m'
C:\Users\bhc02\AppData\Local\Temp\ipykernel_3340\3812164741.py:4: SyntaxWarning: invalid escape sequence '\m'
  df = pd.read_csv("kaggle_dataset\marketing_campaign\marketing_campaign", sep=";")
C:\Users\bhc02\AppData\Local\Temp\ipykernel_3340\3812164741.py:4: SyntaxWarning: invalid escape sequence '\m'
  df = pd.read_csv("kaggle_dataset\marketing_campaign\marketing_campaign", sep=";")


FileNotFoundError: [Errno 2] No such file or directory: 'kaggle_dataset\\marketing_campaign\\marketing_campaign'

In [ ]:
#1. 미사용 컬럼 제거
df = df.drop(columns=['Education', 'Marital_Status', 'Dt_Customer', 'Recency', 'Z_CostContact', 'Z_Revenue'])
df.head()

,ID,Year_Birth,Income,Kidhome,Teenhome,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,5524,1957,58138.0,0,0,635,88,546,172,88,...,10,4,7,0,0,0,0,0,0,1
1,2174,1954,46344.0,1,1,11,1,6,2,1,...,1,2,5,0,0,0,0,0,0,0
2,4141,1965,71613.0,0,0,426,49,127,111,21,...,2,10,4,0,0,0,0,0,0,0
3,6182,1984,26646.0,1,0,11,4,20,10,3,...,0,4,6,0,0,0,0,0,0,0
4,5324,1981,58293.0,1,0,173,43,118,46,27,...,3,6,5,0,0,0,0,0,0,0


In [ ]:
#2. 결측치 처리
df.isna().sum()
    #Income 결측치가 24개 존재
    #-> 제거로 처리
df = df.dropna()
df.isna().sum()

ID                     0
Year_Birth             0
Income                 0
Kidhome                0
Teenhome               0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
Response               0
dtype: int64

In [ ]:
#3. 이상치 처리

# 나이
age_mask = (df['Year_Birth'] > 1930) & (df['Year_Birth'] < 2010)
df = df[age_mask]
# 소득
income_mask = (df['Income'] > 0) & (df['Income'] < 200000)
df = df[income_mask]

In [ ]:
#4. 파생변수 생성
    #컬럼으로 추가 : 영구 파생변수 / 변수로만 사용 : 일시 파생변수
    #나이
df['Age'] = 2020 - df['Year_Birth']
    #총 캠페인 반응 횟수
total_cmp = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response']
df['total_accepted_cmp'] = df[total_cmp].sum(axis=1)

#4.5. (인코딩)
    #문자를 숫자로 바꾸는 작업
    #순서가 있는 데이터 경우 0~n까지 숫자로 매핑 : 라벨인코딩
    #순서가 없는 데이터 경우 더미변수화 : 원핫인코딩
         #ex) single, married, toghether컬럼을 하나씩 만들고 해당되는 칸에 1표시

In [ ]:
#5. 정규화
    #수입 정규화_standard
mean_income = df['Income'].mean()
std_income = df['Income'].std()

standard_income = (df['Income'] - mean_income) / std_income
df['Income'] = standard_income

In [ ]:
#6. 회귀
import statsmodels.api as sm
    
    #(단순회귀1. 캠페인에 1번 이상 반응한 고객은 온라인 구매를 더 많이 할까?)
#종속변수 Y : NumWebPurchases
#독립변수 X : total_accepted_cmp

    #groupby를 이용한 평균 비교
df['CampaignResponse'] = (df['total_accepted_cmp'] > 0).astype(int)
print(df.groupby('CampaignResponse')['NumWebPurchases'].mean(), end="\n\n")
    #groupby() : 캠페인반응이 같은 사람끼리 group 후 웹구매의 평균을 구한다.

CampaignResponse
0    3.717040
1    5.076159
Name: NumWebPurchases, dtype: float64



groupby _ 캠페인에 한번이라도 반응한 사람은 웹 구매 횟수가 
그렇지 않은 사람보다 평균 약 1.36회 더 많다

In [ ]:
    #(단순회귀2. 캠페인에 많이 반응할 수록 온라인 구매가 늘어날까?)
#종속변수 Y : NumWebPurchases
#독립변수 X : total_accepted_cmp

#value_counts() : 각 값이 몇 개 있는지 세기
#sort_index() : 인덱스 순서대로 정렬
df['total_accepted_cmp'].value_counts().sort_index()

    #groupby를 이용한 평균 비교
print(df.groupby('total_accepted_cmp')['NumWebPurchases'].mean())
    #캠페인 반응 횟수별로 group 후 웹구매의 평균을 구한다


    #회귀를 이용한 검증
X1 = df[['total_accepted_cmp']]
X1 = sm.add_constant(X1) 
Y1 = df['NumWebPurchases']

model1 = sm.OLS(Y1, X1).fit()
print(model1.summary())


NameError: name 'df' is not defined

groupby -> 캠페인 반응횟수가 많을수록 구매금액도 커짐을 알 수 있다

결정계수 R^2 : 0.044 / 캠페인 반응 횟수는 온라인 구매 횟수 변동의 약 4.4%를 설명
  - 관계가 있지만 강력하진 않다

회귀계수 coef : X가 1증가할 때 Y의 변화량

const(β0) 절편 : 3.7986 / 캠페인 반응0회 고객의 평균 온라인 구매 횟수는 3.8회

total_accepted_cmp(β1) 회귀계수 : 0.6451
    / 캠페인 반응횟수 1회 증가 할 때 온라인 구매 평균 0.65회 증가

p-value 통계 신뢰성 : 0.00 (<0.05) / 캠페인 반응횟수와 온라인 구매횟수는 유의미함
    신뢰구간 : 0.520 ~ 0.770 /실제 증가량은 0.52 ~ 0.77회 안에 있을 가능성이 높음

F-statistic p-value : 1.74 

t 개별 회귀계수 검정 : 각 회귀계수를 검정한다
  / t = coef / std err(평균오차) -> 0.645 / 0.064 = 10.1
  / 회귀계수는 표준오차 대비 10배이상 크다 -> 우연일 가능성이 매우 낮다
  / p-value만 봐도 우연성 검증 가능

In [ ]:
    #(단순회귀2. 캠페인은 온라인,오프라인,카테고리 중 어디에 더 영향을 미칠까?)
#종속변수 Y : NumWebPurchases | NumCatalogPurchases | NumStorePurchases
#독립변수 X : total_accepted_cmp

X2 = df[['total_accepted_cmp']]
X2 = sm.add_constant(X2)
Y2_web = df['NumWebPurchases']
Y2_catalog = df['NumCatalogPurchases']
Y2_store = df['NumStorePurchases']

model2_web = sm.OLS(Y2_web, X2).fit()
model2_catalog = sm.OLS(Y2_catalog, X2).fit()
model2_store = sm.OLS(Y2_store, X2).fit()
print("웹구매 회귀분석 결과\n", model2_web.summary(), end="\n\n")
print("카탈로그구매 회귀분석 결과\n", model2_catalog.summary(), end="\n\n")
print("매장구매 회귀분석 결과\n", model2_store.summary())

웹구매 회귀분석 결과
                             OLS Regression Results                            
Dep. Variable:        NumWebPurchases   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     102.0
Date:                Mon, 05 Jan 2026   Prob (F-statistic):           1.74e-23
Time:                        23:43:13   Log-Likelihood:                -5319.6
No. Observations:                2212   AIC:                         1.064e+04
Df Residuals:                    2210   BIC:                         1.065e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  

# 회귀결과

#### 웹
- 결정계수 : 0.044
- 절편 : 3.79
- 회귀계수 : 0.6451
- p-value : 0.00

#### 카탈로그
- 결정계수 : 0.123 ★
- 절편 : 2.15
- 회귀계수 : 1.15 ★
- p-value : 0.00

#### 매장
- 결정계수 : 0.028
- 절편 : 5.53
- 회귀계수 : 0.61
- p-value : 0.00

> 캠페인은 카탈로그에 가장 효과를 보임

> 웹, 카탈로그, 매장 모두 캠페인에 유의미한 영향을 받음

In [ ]:
    #(다중회귀1. 수입과 연령대를 통제했을 때 캠페인의 효과검증)

#종속변수 Y : NumWebPurchases
#독립변수 X : total_accepted_cmp, Income, Age

X3 = df[['total_accepted_cmp', 'Income', 'Age']]
X3 = sm.add_constant(X3)
Y3_web = df['NumWebPurchases']
model3_web = sm.OLS(Y3_web, X3).fit()
print("웹구매 다중회귀분석 결과\n", model3_web.summary(), end="\n\n")


웹구매 다중회귀분석 결과
                             OLS Regression Results                            
Dep. Variable:        NumWebPurchases   R-squared:                       0.220
Model:                            OLS   Adj. R-squared:                  0.219
Method:                 Least Squares   F-statistic:                     207.5
Date:                Mon, 05 Jan 2026   Prob (F-statistic):          1.44e-118
Time:                        23:43:13   Log-Likelihood:                -5094.9
No. Observations:                2212   AIC:                         1.020e+04
Df Residuals:                    2208   BIC:                         1.022e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                

# 다중회귀 분석 결과

#### 결정계수 R^2
- 캠페인 반응만 고려했을 때 : 4.4% 설명
- 캠페인&수입&나이 고려했을 때 : 22% 설명

#### 회귀계수 
- 캠페인 반응만 고려했을 때 : 0.645
- 캠페인&수입&나이 고려했을 때 캠페인의 회귀계수: 0.203
##### -> 캠페인 효과만을 남겨 봤을 때 0.203이 진짜 캠페인의 효과